In [4]:
from django.shortcuts import render

# Create your views here.

import csv
import io
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


from django.conf import settings
from django.shortcuts import render
# from .forms import CSVUploadForm
from django.http import HttpResponse

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from django.shortcuts import render
# from .forms import MLForm
from django.shortcuts import render
from django.http import JsonResponse
import json
import time
import pickle
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [5]:
representation = 'bow'
classifier = 'nn'

In [ ]:
df = pd.read_csv("./media/data/IMDB Dataset.csv")

# Split features and labels
X = df['review'].values
y = df['sentiment'].values

# train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Initialize the text representation module based on selection
if representation == 'tfidf':
    vectorizer = TfidfVectorizer(max_features=10000)
    representation_name = "TF-IDF"
elif representation == 'bow':
    vectorizer = CountVectorizer(max_features=10000)
    representation_name = "Bag of Words"
else:
    JsonResponse({'error': 'Invalid representation method'}, status=400)

# Transform the text data
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Initialize the classifier based on selection
if classifier == 'logreg':
    model = LogisticRegression(max_iter=1000)
    classifier_name = "Logistic Regression"
elif classifier == 'svm':
    model = SVC(probability=True)
    classifier_name = "Support Vector Machine"
elif classifier == 'nn':
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300)
    classifier_name = "Neural Network"
else:
    JsonResponse({'error': 'Invalid classifier method'}, status=400)

# Train the model
model.fit(X_train_vec, y_train)

# Save with pickle
vectorizer_path = os.path.join("./project2/models", 'vectorizer.pkl')
model_path = os.path.join("./project2/models", 'classifier.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

with open(vectorizer_path, 'wb') as f:
    pickle.dump(vectorizer, f)

# Evaluate on test set
accuracy = model.score(X_test_vec, y_test)

# Calculate training time
training_time = time.time() - start_time

# Save the model and vectorizer for later use
model_dir = os.path.join('project2', 'models')
os.makedirs(model_dir, exist_ok=True)

with open(os.path.join(model_dir, f'vectorizer_{representation}.pkl'), 'wb') as f:
    pickle.dump(vectorizer, f)

with open(os.path.join(model_dir, f'model_{representation}_{classifier}.pkl'), 'wb') as f:
    pickle.dump(model, f)

# Save metadata about the current model
model_metadata = {
    'representation': representation,
    'classifier': classifier,
    'accuracy': accuracy,
    'representation_name': representation_name,
    'classifier_name': classifier_name
}

with open(os.path.join(model_dir, 'current_model_metadata.json'), 'w') as f:
    json.dump(model_metadata, f)


NameError: name 'models_dir' is not defined